In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
from torchvision import transforms, models

# --- CONFIGURATION ---
BATCH_SIZE = 64
IMG_SIZE = 128
LEARNING_RATE = 1e-4
EPOCHS = 15
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"⚙️ Configuration : {DEVICE} | Batch: {BATCH_SIZE} | Img: {IMG_SIZE}")

In [ ]:
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
from torchvision import transforms
from PIL import Image
import torch

# --- CONFIGURATION ---
# Change this path to your local folder containing the subfolders
DATA_ROOT = "/kaggle/input/autovi-dataset-version-1-0-0/AUTOVI Dataset Version 1.0.0/10459003" 

train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

class UniversalAutoViDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []       
        self.part_names = []   
        self.stratify_labels = [] 
        
        seen_filenames = set()

        for root, dirs, files in os.walk(root_dir):
            png_files = [f for f in files if f.lower().endswith('.png')]
            if not png_files: continue
            
            path_parts = root.split(os.sep)
            # Detect part name from directory structure
            part_name = "unknown"
            if 'train' in path_parts:
                part_name = path_parts[path_parts.index('train') - 1]
            elif 'test' in path_parts:
                part_name = path_parts[path_parts.index('test') - 1]
            else:
                continue 

            if part_name in ['ground_truth', 'images']: continue

            for file in png_files:
                unique_id = f"{part_name}/{file}"
                if unique_id in seen_filenames: continue
                seen_filenames.add(unique_id)

                self.image_paths.append(os.path.join(root, file))
                label = 0 if 'good' in path_parts else 1
                self.labels.append(label)
                self.part_names.append(part_name)
                self.stratify_labels.append(f"{part_name}_{label}")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(self.labels[idx], dtype=torch.long), self.part_names[idx]

# --- INITIALIZATION ---
try:
    full_dataset = UniversalAutoViDataset(DATA_ROOT, transform=None)

    train_idx, test_idx = train_test_split(
        list(range(len(full_dataset))),
        test_size=0.2,
        stratify=full_dataset.stratify_labels, 
        random_state=42
    )

    train_loader = DataLoader(
        UniversalAutoViDataset(DATA_ROOT, transform=train_transform), 
        batch_size=BATCH_SIZE, sampler=SubsetRandomSampler(train_idx)
    )
    test_loader = DataLoader(
        UniversalAutoViDataset(DATA_ROOT, transform=test_transform), 
        batch_size=BATCH_SIZE, sampler=SubsetRandomSampler(test_idx)
    )
    
    print(f"Data loaded: {len(train_idx)} train, {len(test_idx)} test images.")
except Exception as e:
    print(f"Error: {e}")

In [ ]:
class ExpertResNet(nn.Module):
    def __init__(self):
        super(ExpertResNet, self).__init__()
        self.net = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        
        num_features = self.net.fc.in_features
        self.net.fc = nn.Linear(num_features, 2)
        
    def forward(self, x):
        return self.net(x)
    
    def get_last_conv_layer(self):
        return self.net.layer4[-1]

model = ExpertResNet().to(DEVICE)
print("✅ Modèle ResNet18 chargé.")

In [ ]:
train_labels = [full_dataset.labels[i] for i in train_idx]
n_good = train_labels.count(0)
n_defect = train_labels.count(1)
weight = n_good / n_defect if n_defect > 0 else 1.0

print(f"⚖️ Balance Train -> Good: {n_good}, Defect: {n_defect} (Poids: {weight:.2f})")
class_weights = torch.tensor([1.0, weight]).to(DEVICE)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2)

def train_loop():
    train_losses = []
    
    for epoch in range(EPOCHS):
        model.train()
        running_loss = 0.0
        
        for images, labels, _ in train_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        # Validation
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels, _ in test_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = model(images)
                val_loss += criterion(outputs, labels).item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        avg_train_loss = running_loss / len(train_loader)
        avg_val_loss = val_loss / len(test_loader)
        acc = 100 * correct / total
        train_losses.append(avg_train_loss)
        
        lr = optimizer.param_groups[0]['lr']
        print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Acc: {acc:.2f}% | LR: {lr:.1e}")
        
        scheduler.step(avg_val_loss)
        
    return train_losses

history = train_loop()
torch.save(model.state_dict(), "resnet_autovi_final.pth")
print("💾 Modèle sauvegardé.")

In [ ]:
def evaluate_by_part(model, loader):
    model.eval()
    
    y_true = []
    y_pred = []
    y_parts = []
    
    with torch.no_grad():
        for images, labels, parts in loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
            y_parts.extend(parts)
            
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Good', 'Defect'], yticklabels=['Good', 'Defect'])
    plt.title('Matrice de Confusion Globale')
    plt.show()
    
    print("\n--- 📊 Précision par Type de Pièce ---")
    print(f"{'Pièce':<20} | {'Total':<6} | {'Précision':<8}")
    print("-" * 45)
    
    unique_parts = sorted(list(set(y_parts)))
    for part in unique_parts:
        indices = [i for i, p in enumerate(y_parts) if p == part]
        
        part_true = [y_true[i] for i in indices]
        part_pred = [y_pred[i] for i in indices]
        
        if len(indices) > 0:
            correct = sum([1 for t, p in zip(part_true, part_pred) if t == p])
            acc = 100 * correct / len(indices)
            print(f"{part:<20} | {len(indices):<6} | {acc:.2f}%")

print("🔍 Analyse détaillée...")
evaluate_by_part(model, test_loader)

In [ ]:
# --- CLASSE GRAD-CAM ---
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        self.target_layer.register_forward_hook(self.save_activation)
        self.target_layer.register_full_backward_hook(self.save_gradient)

    def save_activation(self, module, input, output):
        self.activations = output

    def save_gradient(self, module, grad_input, grad_output):
        self.gradients = grad_output[0]

    def __call__(self, x, class_idx=None):
        output = self.model(x)
        if class_idx is None:
            class_idx = torch.argmax(output)
            
        self.model.zero_grad()
        score = output[0, class_idx]
        score.backward()
        
        gradients = self.gradients[0]
        pooled_gradients = torch.mean(gradients, dim=[1, 2])
        activations = self.activations[0]
        
        for i in range(activations.shape[0]):
            activations[i, :, :] *= pooled_gradients[i]
            
        heatmap = torch.mean(activations, dim=0).cpu().detach().numpy()
        heatmap = np.maximum(heatmap, 0)
        if np.max(heatmap) != 0:
            heatmap /= np.max(heatmap)
        return heatmap

def show_cam(model, dataset, idx):

    img_pil, lbl, part_name = dataset[idx]
    
    input_tensor = test_transform(img_pil).unsqueeze(0).to(DEVICE)
    
    target_layer = model.get_last_conv_layer()
    grad_cam = GradCAM(model, target_layer)
    
    heatmap = grad_cam(input_tensor, class_idx=1)
    
    plt.figure(figsize=(10, 4))
    
    img_disp = np.array(img_pil) 
    
    plt.subplot(1, 2, 1)
    plt.imshow(img_disp)
    plt.title(f"Pièce: {part_name}\nLabel: {'Defect' if lbl==1 else 'Good'}")
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    heatmap = cv2.resize(heatmap, (img_disp.shape[1], img_disp.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
    
    superimposed = cv2.addWeighted(img_disp, 0.6, heatmap, 0.4, 0)
    
    plt.imshow(superimposed)
    plt.title("Grad-CAM Focus")
    plt.axis('off')
    plt.show()

# --- TEST ---
print("📸 Visualisation d'un défaut aléatoire...")
defect_indices = [i for i, x in enumerate(full_dataset.labels) if x == 1]

if defect_indices:
    import random
    for _ in range(3):
        idx = random.choice(defect_indices)
        show_cam(model, full_dataset, idx)
else:
    print("Aucun défaut trouvé dans le dataset.")

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(history, label='Training Loss')
plt.title('Courbe d\'apprentissage (ResNet18)')
plt.xlabel('Époques')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()